In [1]:
import numpy as np
import matplotlib.pyplot as plt

In [5]:
# Load files
raw_obs = np.loadtxt(r"C:\Users\paulv\Documents\SCHOOLWORK\CMPE 195B Senior Design Project II\GSE63525_GM12878_primary_intrachromosomal_contact_matrices\GM12878_primary\1mb_resolution_intrachromosomal\chr1\MAPQGE30/chr1_1mb.RAWobserved") # 2v2 matrix, value is num count
raw_exp = np.loadtxt(r"C:\Users\paulv\Documents\SCHOOLWORK\CMPE 195B Senior Design Project II\GSE63525_GM12878_primary_intrachromosomal_contact_matrices\GM12878_primary\1mb_resolution_intrachromosomal\chr1\MAPQGE30/chr1_1mb.RAWexpected").ravel() #1d vector

raw_obs.shape, raw_exp.shape

((26563, 3), (249,))

In [13]:
bin_size = 1_000_000

i = (raw_obs[:,0] // bin_size).astype(int) #bin indices
j = (raw_obs[:,1] // bin_size).astype(int)
v = raw_obs[:,2].astype(float)

n = int(max(i.max(), j.max())) + 1   # number of bins (expect ~250 for chr1 at 1Mb)
i.shape, j.shape, v.shape, n

((26563,), (26563,), (26563,), 250)

In [7]:
# Build dense with accumulation
obs = np.zeros((n, n), dtype=float)
np.add.at(obs, (i, j), v)
np.add.at(obs, (j, i), v)            # symmetrize (in case file is upper-tri only)

# optional: ensure diagonal present
obs[np.arange(n), np.arange(n)] += 0
obs

array([[1.29014e+05, 4.63540e+04, 4.06500e+03, ..., 1.72000e+02,
        9.00000e+01, 1.36000e+02],
       [4.63540e+04, 7.69174e+05, 5.92460e+04, ..., 4.70000e+02,
        2.36000e+02, 2.68000e+02],
       [4.06500e+03, 5.92460e+04, 7.03508e+05, ..., 5.09000e+02,
        2.62000e+02, 1.55000e+02],
       ...,
       [1.72000e+02, 4.70000e+02, 5.09000e+02, ..., 5.90880e+05,
        3.52770e+04, 8.65400e+03],
       [9.00000e+01, 2.36000e+02, 2.62000e+02, ..., 3.52770e+04,
        2.46512e+05, 2.76990e+04],
       [1.36000e+02, 2.68000e+02, 1.55000e+02, ..., 8.65400e+03,
        2.76990e+04, 1.07312e+05]], shape=(250, 250))

In [8]:
exp_full = np.r_[np.nan, raw_exp]                     # now length should be n (=250)
exp_full

array([         nan, 281994.12   ,  64373.23   ,  18104.424  ,
        10357.301  ,   7389.737  ,   5805.942  ,   4859.497  ,
         4246.0547 ,   3820.177  ,   3481.5405 ,   3253.6357 ,
         3059.8591 ,   2882.885  ,   2742.5476 ,   2630.8936 ,
         2535.0496 ,   2444.054  ,   2364.6108 ,   2268.2725 ,
         2181.877  ,   2110.2493 ,   2045.44   ,   2004.358  ,
         1945.6239 ,   1881.2086 ,   1837.7605 ,   1811.9805 ,
         1785.4465 ,   1741.8208 ,   1705.2538 ,   1668.6223 ,
         1615.6288 ,   1555.6903 ,   1508.4504 ,   1466.6456 ,
         1428.3175 ,   1385.0204 ,   1361.2499 ,   1343.4977 ,
         1294.964  ,   1252.6713 ,   1222.2277 ,   1209.967  ,
         1197.1888 ,   1165.3282 ,   1132.8535 ,   1100.1016 ,
         1071.6692 ,   1033.9835 ,   1003.30865,    976.9369 ,
          959.652  ,    933.2144 ,    920.82324,    902.0525 ,
          880.0685 ,    864.06165,    847.99304,    837.33966,
          816.47516,    807.2361 ,    799.1017 ,    777

In [9]:
# Build expected matrix by distance and compute O/E 
idx_i = np.arange(n)[:, None] # column vector
idx_j = np.arange(n)[None, :] # row vector
dist = np.abs(idx_i - idx_j)  # values in 0..n-1

dist

array([[  0,   1,   2, ..., 247, 248, 249],
       [  1,   0,   1, ..., 246, 247, 248],
       [  2,   1,   0, ..., 245, 246, 247],
       ...,
       [247, 246, 245, ...,   0,   1,   2],
       [248, 247, 246, ...,   1,   0,   1],
       [249, 248, 247, ...,   2,   1,   0]], shape=(250, 250))

In [10]:
exp_mat = exp_full[dist]   
exp_mat

array([[         nan, 281994.12   ,  64373.23   , ...,    359.3574 ,
           291.8051 ,    318.23862],
       [281994.12   ,          nan, 281994.12   , ...,    424.49094,
           359.3574 ,    291.8051 ],
       [ 64373.23   , 281994.12   ,          nan, ...,    439.10712,
           424.49094,    359.3574 ],
       ...,
       [   359.3574 ,    424.49094,    439.10712, ...,          nan,
        281994.12   ,  64373.23   ],
       [   291.8051 ,    359.3574 ,    424.49094, ..., 281994.12   ,
                 nan, 281994.12   ],
       [   318.23862,    291.8051 ,    359.3574 , ...,  64373.23   ,
        281994.12   ,          nan]], shape=(250, 250))

In [11]:
OE = obs / exp_mat
OE[~np.isfinite(OE)] = np.nan           # clean inf/NaN
np.fill_diagonal(OE, np.nan)            # ignore self-self
OE

array([[       nan, 0.16437931, 0.06314737, ..., 0.47863214, 0.30842504,
        0.42735228],
       [0.16437931,        nan, 0.21009658, ..., 1.10720855, 0.65672781,
        0.91842123],
       [0.06314737, 0.21009658,        nan, ..., 1.15917045, 0.61720987,
        0.43132547],
       ...,
       [0.47863214, 1.10720855, 1.15917045, ...,        nan, 0.12509835,
        0.13443476],
       [0.30842504, 0.65672781, 0.61720987, ..., 0.12509835,        nan,
        0.09822545],
       [0.42735228, 0.91842123, 0.43132547, ..., 0.13443476, 0.09822545,
               nan]], shape=(250, 250))

In [12]:
p = 95 # threshold percentile  
tau = np.nanpercentile(OE, p) # threshold value
A = (OE >= tau).astype(int)
A[np.isnan(OE)] = 0
np.fill_diagonal(A, 0)

A

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], shape=(250, 250))